## Task

Given a number $N$ and some lists of integers $P = (L_0, L_1, L_2, ..., L_n)$, 
determine is possible $S = (L_{s_0}, L_{s_1}, L_{s_2}, ..., L_{s_n})$
such that each number between $0$ and $N-1$ appears in at least one list

$$\forall n \in [0, N-1] \ \exists i : n \in L_{s_i}$$

and that the total numbers of elements in all $L_{s_i}$ is minimum. 

## Instructions

* Create the directory `lab1` inside the course repo (the one you registered with Andrea)
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)
* Use `problem` to generate the problems with different $N$
* In the `README.md`, report the the total numbers of elements in $L_{s_i}$ for problem with $N \in [5, 10, 20, 100, 500, 1000]$ and the total number on $nodes$ visited during the search. Use `seed=42`.
* Use `GitHub Issues` to peer review others' lab

In [2]:
import random
import logging
import itertools
import numpy as np
import tqdm
from collections import Counter
from gx_utils import *
from typing import Callable
from copy import copy

In [3]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

## Greedy solution
Does not find the optimal solution for N greater than 5 but the computation is feasible both in terms of visited nodes and computation time

In [3]:
def set_cover(P,N):
    goal=set(range(0,N))
    n_nodes=0
    P_sets=[set(x) for x in P]
    lengths=[len(x) for x in P_sets]
    elements=set(e for s in P for e in s)
    if elements!=goal:
        return None
        
    covered=set()
    S=[]
    while covered!=elements:
        subset=max(P_sets, key=lambda s: len(s-covered)/lengths[P_sets.index(s)])
        n_nodes+=1
        S.append(subset)
        covered |= subset
    w=sum(len(_) for _ in S)
    logging.getLogger().setLevel(logging.INFO)
    logging.info(f" Optimized solution for N={N}: w={w} (bloat={int((w-N)*100/N)}%), nodes visited={n_nodes}")
    return S, n_nodes

In [4]:
N=10
SEED=42
solution, n_nodes=set_cover(problem(N,SEED),N)
print(solution)
print(n_nodes)

INFO:root: Optimized solution for N=10: w=12 (bloat=20%), nodes visited=5


[{0, 4}, {1, 2, 3}, {9, 6}, {2, 5, 7}, {8, 3}]
5


In [5]:
SEED=42
for n in [5,10,20,100,500,1000]:
    set_cover(problem(n,SEED),n)

INFO:root: Optimized solution for N=5: w=5 (bloat=0%), nodes visited=5
INFO:root: Optimized solution for N=10: w=12 (bloat=20%), nodes visited=5
INFO:root: Optimized solution for N=20: w=30 (bloat=50%), nodes visited=6
INFO:root: Optimized solution for N=100: w=171 (bloat=71%), nodes visited=8
INFO:root: Optimized solution for N=500: w=1256 (bloat=151%), nodes visited=12
INFO:root: Optimized solution for N=1000: w=2913 (bloat=191%), nodes visited=13


## Combinations
Finds the optimal solution for N=[5,10,20]. For N greater than 20 it results in a memory allocation error

In [6]:
def combinations_search(P,N):
    '''for every combination of lists in P check if it's a solution. If it is save it and at the end compare all solutions'''
    n_nodes=0
    solutions=dict()
    universe=set(range(0,N))
    avg_len=sum(len(el) for el in P)/len(P) #average length of the lists generated by problem
    avg_subsets=int(N//avg_len+N/5) #estimate that the optimal solution will be found considering a number of lists close to N/avg_len
    for i in range(2,avg_subsets):
        temp=list(itertools.combinations(P,i))
        temp=list(list(el) for el in temp)  
        n_nodes+=len(temp)
        for el in temp:
            current_elements=set(e for l in el for e in l)
            if current_elements==universe:
                solutions[sum(len(_) for _ in el)]=el
    len_sol=min(list(solutions.keys()))
    logging.getLogger().setLevel(logging.INFO)
    logging.info(f" Optimized solution for N={N}: w={len_sol}, nodes visited={n_nodes}")
    return solutions[len_sol], len_sol, n_nodes

In [7]:
N=10
SEED=42
solution, len_sol, n_nodes=combinations_search(problem(N,SEED),N)
print(solution)
print(n_nodes)

INFO:root: Optimized solution for N=10: w=10, nodes visited=251125


[[1, 7], [8, 2], [4, 5, 6], [0, 9, 3]]
251125


In [8]:
SEED=42
for n in [5,10,20]:
    combinations_search(problem(n,SEED),n)

INFO:root: Optimized solution for N=5: w=5, nodes visited=2600
INFO:root: Optimized solution for N=10: w=10, nodes visited=251125
INFO:root: Optimized solution for N=20: w=23, nodes visited=1676081


## Graph Search
Finds the optimal solution but for N greater than 20 the execution time is excessive

In [10]:
from gx_utils import *
import logging
import random
from typing import Callable

In [11]:
class State:
    def __init__(self, data: set):
        self._data = set(data)

    def __hash__(self):
        return hash(frozenset(self._data))

    def __eq__(self, other):
        return self._data == other._data

    def __lt__(self, other):
        return self._data < other._data

    def __or__(self, other):
        return State(self._data | other._data)

    def __and__(self, other):
        return State(self._data & other._data)

    def __sub__(self, other):
        return State(self._data - other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)  

    def __len__(self):
        return len(self._data)        

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [12]:
def search( 
    initial_state: State,
    GOAL: State,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable
):
    parent_state.clear()
    state_cost.clear()
    frontier = PriorityQueue()
    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0
    generated_states = 0
    visited_states = 1

    while state is not None and state!=GOAL:
        visited_states += 1
        for a in [State(x) for x in P]:            
            new_state = state.__or__(a)
            cost = unit_cost(state, a)
            generated_states += 1

            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = (state, a)
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                logging.debug(f"Added new node {new_state} to frontier (cost={state_cost[new_state]}, h = {priority_function(new_state)})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                parent_state[new_state] = (state, a)
                state_cost[new_state] = state_cost[state] + cost
                logging.debug(f"Updated node {new_state} cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while parent_state[s]:
        s, a = parent_state[s]
        path.append(a)

    logging.info(f"Found a solution with {sum(len(_.data) for _ in path)} elements; visited {visited_states} states over {generated_states} generated states")
    return list(reversed(path))

In [13]:
logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20]:
    parent_state = dict()
    state_cost = dict()
    GOAL = State(set(range(N)))
    P = problem(N, seed=42) 
    INITIAL_STATE = State(set())
    logging.info(f'N = {N}')

    final = search(
        INITIAL_STATE,
        GOAL,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: state_cost[s],
        unit_cost=lambda state, action: len(state & action),
    )
    logging.debug(final)

INFO:root:N = 5
INFO:root:Found a solution with 5 elements; visited 32 states over 775 generated states
INFO:root:N = 10
INFO:root:Found a solution with 10 elements; visited 583 states over 29100 generated states
INFO:root:N = 20
INFO:root:Found a solution with 23 elements; visited 2864 states over 97342 generated states


# 
<span style="color:red">**Warning:**</span> everything written from this point forward has been added after the 16/10/2022 deadline and it is taken from the professor's Squillero 2021-22 folder. I'm editing a few things, adding some functions and adapting the code to this year's problem.

In [131]:
def is_valid(N,solution):
    universe = set(range(0,N))
    sol = set(el for list_ in solution for el in list_)
    return universe==sol

def p_unique(P):
    '''function that removes duplicates from P (list of lists)'''
    #there is definitely a better way to do this...
    P_sets=[frozenset(_) for _ in P]
    temp=set()
    result=list()
    for el in P_sets:
        temp.add(el)
    temp=list(temp)
    for el in temp:
        result.append(list(el))
    return result

def solution_cost(solution): #the cost is the number of numbers in a solution
    return sum(len(el) for el in solution)

In [82]:
def tweak(solution):
    new_solution = solution.copy()
    index = None
    while index is None or np.random.random() < 0.2:
        index = np.random.randint(0, N)
        new_solution[index] = not new_solution[index]
    return new_solution

# Vanilla Hill Climber

In [195]:
def hill_climber(N, P, MAX_STEPS):
    P = set(tuple(_) for _ in P)
    universe = set(range(N))

    def solution_cost(solution):
        cnt=Counter()
        cnt.update(sum((e for e in solution), start=()))
        return len(cnt), -sum(len(el) for el in solution)

    def is_valid(solution):
        return universe == set(_ for _ in solution)

    def tweak(solution):
        new_solution=set(solution)
        while new_solution and random.random() < 0.7:   #if solution is not empty and random.random is < 0.7 remove randomly one tuple in the solution
            new_solution.remove(random.choice(list(new_solution)))
        while random.random() < 0.7:    #with prob=0.7 add to the solution a new list (that is not already in solution)
            new_solution.add(random.choice(list(P - solution)))
        return new_solution

    current_solution=set() #initialize the solution to an empty set
    useless_steps=0
    while useless_steps < MAX_STEPS and not is_valid(solution):
        useless_steps += 1
        candidate_solution = tweak(current_solution)
        if solution_cost(candidate_solution) > solution_cost(current_solution):
            useless_steps = 0
            current_solution = copy(candidate_solution)
            logging.debug(f"New solution: cost={solution_cost(current_solution)}")
    logging.info(f"Solution for N={N}: w={sum(len(_) for _ in current_solution)} (bloat={(sum(len(_) for _ in current_solution)-N)/N*100:.0f}%)")
    return current_solution

In [198]:
logging.getLogger().setLevel(logging.INFO)
MAX_STEPS = 10_000
SEED = 42
np.random.seed(SEED)
for N in [5,10,20,100,500,1000]:
    hill_climber(N, problem(N,SEED), MAX_STEPS)

INFO:root:Solution for N=5: w=5 (bloat=0%)
INFO:root:Solution for N=10: w=11 (bloat=10%)
INFO:root:Solution for N=20: w=24 (bloat=20%)
INFO:root:Solution for N=100: w=214 (bloat=114%)
INFO:root:Solution for N=500: w=1566 (bloat=213%)
INFO:root:Solution for N=1000: w=3383 (bloat=238%)
